In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
# from sklearn.tree import DecisionTreeClassifier 

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, info_gain=None, value=None , children = []):
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.children = children
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value
    def add_child(self, child):
        self.children.append(child)
        
    def __str__(self):
        if self.value is  not None:
            res = "F_"+self.feature_index+", treshold: "+self.threshold+", IG:"+self.info_gain+", Children:\n\t"+self.left+"\n\t"+self.right
            return res
        res = self.value 
        return res

In [3]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2 , n_children = 3):
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
        self.n_children = n_children
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            
            step = (np.max(possible_thresholds)  - np.min(possible_thresholds)) // self.n_children
            min_val = np.min(possible_thresholds)
            thresholds = [min_val + step * i for i in range(1, self.n_children)]
            
            # loop over all the feature values present in the data
            # for threshold in possible_thresholds:
                # get current split
            dataset_left, dataset_right = self.split(dataset, feature_index, thresholds)
            children_datasets = self.split(dataset , feature_index , thresholds)
            # check if childs are not null
            # if len(dataset_left)>0 and len(dataset_right)>0:
            y= dataset[:, -1]
            # left_y, right_y  =  dataset_left[:, -1], dataset_right[:, -1]
            children_y = [children_datasets[i][:, -1] for i in range(len(children_datasets))] 
            # compute information gain
            # curr_info_gain = self.information_gain(y, left_y, right_y)
            curr_info_gain = self.information_gain(y,children=  children_y)
            # update the best split if needed
            if curr_info_gain>max_info_gain:
                best_split["feature_index"] = feature_index
                best_split["threshold"] = thresholds
                best_split["dataset_left"] = dataset_left
                best_split["dataset_right"] = dataset_right
                best_split["info_gain"] = curr_info_gain
                max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, thresholds):
        ''' function to split the data '''
        # dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        # dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        children_datasets = [np.array([row for row in dataset if row[feature_index]<=thresholds[0]])]
        for i in range(1 , len(thresholds)):
            temp_child = np.array([row for row in dataset if row[feature_index]<=thresholds[i] and row[feature_index]>thresholds[i-1]])
            children_datasets.append(temp_child)
        children_datasets.append(np.array([row for row in dataset if row[feature_index]>thresholds[-1]]))
        # return dataset_left, dataset_right
        return children_datasets
    
    def information_gain(self, parent, children):
        ''' function to compute information gain
            IG = E(parent) - (weight_l*E(l_child) + weight_r*E(r_child))
        '''
        # weight_l = len(l_child) / len(parent)
        # weight_r = len(r_child) / len(parent)
        children_weights = [len(child)/len(parent) for child in children]
        childrens_entropy = [self.entropy(child) for child in children]
        gain = self.entropy(parent) 
                                    #    *(weight_l *self.entropy(l_child) + weight_r*self.entropy(r_child))
        for i in len(children):
            gain -= children_weights[i] * childrens_entropy[i]
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy 
            E(X) = -p(x)log2(p(x))
        '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("F_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
            # print(str(tree))
            
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [4]:
col_names = ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","Pedigree","Age", "outcome"]
data = pd.read_csv("diabetes.csv", header=0, names=col_names)
# data = pd.read_csv("Resturant.csv", header=0)
# data.head(10)

In [5]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.8)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1)

In [6]:
classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=10)
classifier.fit(X_train,Y_train)
classifier.print_tree()

ValueError: too many values to unpack (expected 2)

In [17]:
Y_pred = classifier.predict(X_test) 
accuracy_score(Y_test, Y_pred)

0.6764227642276422

In [34]:
# X
# Y
# Y_train
# Y_test
# X_train
# X_test